In [3]:
import os
import json
import time
import docx
import fitz  # PyMuPDFをインポート ▼▼▼
import google.generativeai as genai
from dotenv import load_dotenv

# --- 準備 ---
load_dotenv()
try:
    api_key = os.environ.get("GOOGLE_API_KEY")
    if not api_key:
        raise ValueError("APIキーが.envファイルに設定されていません。")
    genai.configure(api_key=api_key)
except ValueError as e:
    print(e)
    exit()

# --- 関数定義 ---

def get_text_from_docx(filepath):
    """docxファイルからテキストを抽出する"""
    try:
        doc = docx.Document(filepath)
        return '\n'.join([para.text for para in doc.paragraphs])
    except Exception as e:
        print(f"  -> エラー: {filepath} の読み込みに失敗 - {e}")
        return ""

# ▼▼▼ PDFからテキストを抽出する関数を追加 ▼▼▼
def get_text_from_pdf(filepath):
    """PDFファイルからテキストを抽出する"""
    try:
        doc = fitz.open(filepath)
        text = ""
        for page in doc:
            text += page.get_text()
        doc.close()
        return text
    except Exception as e:
        print(f"  -> エラー: {filepath} の読み込みに失敗 - {e}")
        return ""

def get_full_text(filepath):
    """ファイルの拡張子に応じて適切な関数でテキストを抽出する"""
    if filepath.lower().endswith('.docx'):
        return get_text_from_docx(filepath)
    elif filepath.lower().endswith('.pdf'):
        return get_text_from_pdf(filepath)
    else:
        print(f"  -> 非対応のファイル形式です: {filepath}")
        return ""

def cosine_similarity(vec_a, vec_b):
    """2つのベクトル間のコサイン類似度を計算する"""
    dot_product = sum(a * b for a, b in zip(vec_a, vec_b))
    norm_a = sum(a * a for a in vec_a) ** 0.5
    norm_b = sum(b * b for b in vec_b) ** 0.5
    if norm_a == 0 or norm_b == 0: return 0
    return dot_product / (norm_a * norm_b)

def get_comparative_analysis(base_text, related_text):
    """2つの論文を比較分析させ、結果をJSONで受け取る"""
    model = genai.GenerativeModel('gemini-2.5-flash')
    # プロンプト内容は変更なし
    prompt = f"""
    あなたは経験豊富な上級研究員です。
    これから渡す2つの論文（基準論文と関連論文）を比較分析し、以下の4つの項目について、具体的な解説をJSON形式で出力してください。

    # 分析項目
    1.  comparison_summary: 
        両論文が取り組む共通の課題と、それぞれの立ち位置の違いを簡潔に要約してください。

    2.  method_comparison: 
        両論文で採用されている手法、アルゴリズム、または実験計画の違いを具体的に説明してください。例えば、検知率などの性能指標に違いがある場合、その原因となりうるアプローチの違いを推測してください。

    3.  result_comparison: 
        両論文の結果や性能を比較してください。どちらがどのような指標で優れているか、あるいはどのようなトレードオフの関係にあるかを客観的に記述してください。

    4.  evolution_analysis: 
        基準論文に対して、関連論文はどのような点で「進歩」または「コンセプトの変化」を遂げているか考察してください。
        ここでの「進歩」とは、単なる性能向上（例：検知率90%）だけを指すものではありません。
        研究の視点を変えたり（例：大きなジェスチャーの研究から、微細なジェスチャーの研究へ）、特定の問題設定に特化したりするなど、一見、限定的になったように見えても、それが新しい価値や視点を生み出している場合は、重要な「進歩」または「変化」として評価してください。

    # 論文本文
    ## 基準論文
    {base_text}

    ## 関連論文
    {related_text}

    # 出力形式 (JSONのみ)
    {{
      "comparison_summary": "（ここに要約）",
      "method_comparison": "（ここに手法の比較）",
      "result_comparison": "（ここに結果の比較）",
      "evolution_analysis": "（ここに進化・変化の考察）"
    }}
    """
    try:
        response = model.generate_content(prompt)
        cleaned_text = response.text.strip().replace('```json', '').replace('```', '').strip()
        return json.loads(cleaned_text)
    except Exception as e:
        print(f"  -> 比較分析中にエラー: {e}")
        return None

# --- 実行部分 ---
input_json_path = 'papers_with_eval.json'
output_json_path = 'final_comparison_data.json'
papers_dir = './papers'

with open(input_json_path, 'r', encoding='utf-8') as f:
    all_papers = json.load(f)

final_data = {}
if os.path.exists(output_json_path):
    with open(output_json_path, 'r', encoding='utf-8') as f:
        final_data = json.load(f)

print("全論文ペアの比較分析を開始します。完了まで長時間かかります...")

for base_paper in all_papers:
    base_filename = base_paper['filename']
    
    if base_filename in final_data:
        print(f"基準論文「{base_filename}」は分析済みのためスキップします。")
        continue

    print(f"基準論文「{base_filename}」の分析を開始...")
    
    other_papers = [p for p in all_papers if p['filename'] != base_filename]
    
    for paper in other_papers:
        paper['score'] = cosine_similarity(base_paper['embedding'], paper['embedding'])
    
    top_3_similar = sorted(other_papers, key=lambda x: x.get('score', 0), reverse=True)[:3]
    
    final_data[base_filename] = []
    
    # ▼▼▼ 統一された関数で基準論文のテキストを取得 ▼▼▼
    base_text = get_full_text(os.path.join(papers_dir, base_paper['filename']))

    for related_paper in top_3_similar:
        print(f"  -> 「{related_paper['filename']}」との比較を生成中...")
        
        # ▼▼▼ 統一された関数で関連論文のテキストを取得 ▼▼▼
        related_text = get_full_text(os.path.join(papers_dir, related_paper['filename']))
        
        if base_text and related_text:
            analysis = get_comparative_analysis(base_text, related_text)
            comparison_result = {
                "compared_to": related_paper['filename'],
                "similarity_score": related_paper.get('score', 0),
                "analysis": analysis
            }
            final_data[base_filename].append(comparison_result)
        
        time.sleep(1) # APIレート制限のため1秒待機

# 全ての分析結果を一つのファイルに保存
with open(output_json_path, 'w', encoding='utf-8') as f:
    json.dump(final_data, f, ensure_ascii=False, indent=2)

print(f"\n全ての比較分析が完了し、'{output_json_path}'に保存しました。")

全論文ペアの比較分析を開始します。完了まで長時間かかります...
基準論文「02_内山田湖太_卒論_最終版.docx」の分析を開始...
  -> 「hujii.pdf」との比較を生成中...
  -> 「kaku.docx」との比較を生成中...
  -> 「CV_20250708_2024000371.pdf」との比較を生成中...
基準論文「2005 matsumoto yoo hirayama petrova.pdf」の分析を開始...
  -> 「CV_20250708_2024000371.pdf」との比較を生成中...
  -> 「hujii.pdf」との比較を生成中...
  -> 「The Japanese Journal of Health Psychology, 2007, Vol.20, No.2, 30-41.pdf」との比較を生成中...
基準論文「CV_20250708_2024000371.pdf」の分析を開始...
  -> 「hujii.pdf」との比較を生成中...
  -> 「The Japanese Journal of Health Psychology, 2007, Vol.20, No.2, 30-41.pdf」との比較を生成中...
  -> 「02_内山田湖太_卒論_最終版.docx」との比較を生成中...
  -> 比較分析中にエラー: Unterminated string starting at: line 5 column 25 (char 4722)
基準論文「EI_16_田中慶人_卒論v4.docx」の分析を開始...
  -> 「The Japanese Journal of Health Psychology, 2007, Vol.20, No.2, 30-41.pdf」との比較を生成中...
  -> 「ito.pdf」との比較を生成中...
  -> 「CV_20250708_2024000371.pdf」との比較を生成中...
  -> 比較分析中にエラー: Extra data: line 6 column 1 (char 1721)
基準論文「hujii.pdf」の分析を開始...
  -> 「CV_20250708_2024000371.pdf」との比較を生成中...
  -> 「02_

In [1]:
import os
import json
import docx
import google.generativeai as genai
from dotenv import load_dotenv

# --- 準備 ---
# .envファイルから環境変数を読み込む
load_dotenv()
# GoogleのAPIキーを設定
try:
    api_key = os.environ.get("GOOGLE_API_KEY")
    if not api_key:
        raise ValueError("APIキーが.envファイルに設定されていません。")
    genai.configure(api_key=api_key)
except ValueError as e:
    print(e)
    # exit() # Jupyter環境ではexit()はカーネルを停止させる可能性があるのでコメントアウト

# --- 関数定義 ---

def get_text_from_docx(filepath):
    """docxファイルからテキストを抽出する"""
    try:
        doc = docx.Document(filepath)
        return '\n'.join([para.text for para in doc.paragraphs])
    except Exception as e:
        print(f"エラー: {filepath} の読み込みに失敗 - {e}")
        return None

def cosine_similarity(vec_a, vec_b):
    """2つのベクトル間のコサイン類似度を計算する"""
    dot_product = sum(a * b for a, b in zip(vec_a, vec_b))
    norm_a = sum(a * a for a in vec_a) ** 0.5
    norm_b = sum(b * b for b in vec_b) ** 0.5
    if norm_a == 0 or norm_b == 0:
        return 0
    return dot_product / (norm_a * norm_b)

def get_comparative_analysis(base_text, related_text):
    """2つの論文を比較分析させ、結果をJSONで受け取る"""
    model = genai.GenerativeModel('gemini-1.5-flash')
    
    prompt = f"""
    あなたは経験豊富な上級研究員です。
    これから渡す2つの論文（基準論文と関連論文）を比較分析し、以下の4つの項目について、具体的な解説をJSON形式で出力してください。

    # 分析項目
    1.  comparison_summary: 
        両論文が取り組む共通の課題と、それぞれの立ち位置の違いを簡潔に要約してください。

    2.  method_comparison: 
        両論文で採用されている手法、アルゴリズム、または実験計画の違いを具体的に説明してください。例えば、検知率などの性能指標に違いがある場合、その原因となりうるアプローチの違いを推測してください。

    3.  result_comparison: 
        両論文の結果や性能を比較してください。どちらがどのような指標で優れているか、あるいはどのようなトレードオフの関係にあるかを客観的に記述してください。

    4.  evolution_analysis: 
        基準論文に対して、関連論文はどのような点で「進歩」または「コンセプトの変化」を遂げているか考察してください。
        ここでの「進歩」とは、単なる性能向上（例：検知率90%）だけを指すものではありません。
        研究の視点を変えたり（例：大きなジェスチャーの研究から、微細なジェスチャーの研究へ）、特定の問題設定に特化したりするなど、一見、限定的になったように見えても、それが新しい価値や視点を生み出している場合は、重要な「進歩」または「変化」として評価してください。

    # 論文本文
    ## 基準論文
    {base_text}

    ## 関連論文
    {related_text}

    # 出力形式 (JSONのみ)
    {{
      "comparison_summary": "（ここに要約）",
      "method_comparison": "（ここに手法の比較）",
      "result_comparison": "（ここに結果の比較）",
      "evolution_analysis": "（ここに進化・変化の考察）"
    }}
    """
    try:
        response = model.generate_content(prompt)
        cleaned_text = response.text.strip().replace('```json', '').replace('```', '').strip()
        return json.loads(cleaned_text)
    except Exception as e:
        print(f"比較分析中にエラー: {e}")
        return None

# --- 実行部分 ---

# ▼▼▼ ここで比較したい基準論文のファイル名を直接指定してください ▼▼▼
base_filename = "02_内山田湖太_卒論_最終版.docx"

json_path = 'papers_with_eval.json'
papers_dir = './papers'

if not os.path.exists(json_path):
    print(f"エラー: データファイル '{json_path}' が見つかりません。")
else:
    with open(json_path, 'r', encoding='utf-8') as f:
        all_papers = json.load(f)

    base_paper = next((p for p in all_papers if p['filename'] == base_filename), None)
    
    if not base_paper:
        print(f"エラー: '{base_filename}' が見つかりません。")
    else:
        # 基準論文以外の論文リスト
        other_papers = [p for p in all_papers if p['filename'] != base_filename]

        # 類似度を計算
        for paper in other_papers:
            paper['score'] = cosine_similarity(base_paper['embedding'], paper['embedding'])
        
        # 最も類似度が高い論文を選ぶ
        if other_papers:
            most_similar_paper = sorted(other_papers, key=lambda x: x.get('score', 0), reverse=True)[0]
            
            print(f"基準論文: {base_paper['filename']}")
            print(f"最も関連性の高い論文: {most_similar_paper['filename']} (スコア: {most_similar_paper.get('score', 0):.3f})")
            print("\n2つの論文の比較分析を開始します...")

            # それぞれの論文の全文テキストを取得
            base_text = get_text_from_docx(os.path.join(papers_dir, base_paper['filename']))
            related_text = get_text_from_docx(os.path.join(papers_dir, most_similar_paper['filename']))

            if base_text and related_text:
                analysis = get_comparative_analysis(base_text, related_text)
                if analysis:
                    print("\n--- 比較分析結果 ---")
                    print(json.dumps(analysis, indent=2, ensure_ascii=False))
        else:
            print("比較対象となる他の論文がありません。")

C:\Users\kotas\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


基準論文: 02_内山田湖太_卒論_最終版.docx
最も関連性の高い論文: kaku.docx (スコア: 0.857)

2つの論文の比較分析を開始します...

--- 比較分析結果 ---
{
  "comparison_summary": "両論文は、それぞれ異なるスポーツ種目において、熟練者と非熟練者のパフォーマンスの違いを明らかにすることを目的としている。基準論文はサッカー選手の状況把握、判断、位置推定能力に焦点を当て、視覚情報処理能力を主に検証している。一方、関連論文は長距離走における最適なピッチ数をランナーのレベル別に分析し、ランニング効率と生理的指標との関係を明らかにすることを目的としている。共通点は、どちらもスポーツにおけるパフォーマンス向上に寄与する要素を、客観的な指標を用いて分析している点である。ただし、基準論文は熟練者と非熟練者の能力差に着目し、関連論文はランナーレベルと最適ピッチ数の関係性に着目するなど、アプローチの仕方が異なる。",
  "method_comparison": "基準論文は、プロサッカー選手の視点動画を用いた位置推定実験と、Go/No-go課題を応用したサッカー場面における意思決定反応実験の2つの実験を実施している。実験1では、動画視聴後の選手位置プロットの正確性と時間を測定し、実験2では、脳波計測を通してN2成分のピーク時間と大きさを測定している。一方、関連論文は、トレッドミル走行中のランナーに対し、モーションキャプチャシステムを用いたピッチ数計測、心拍計とガス分析装置による生理的指標（心拍数、酸素摂取量）の測定という、比較的シンプルな実験方法を採用している。基準論文は多様な手法（動画分析、脳波計測）を用いて複雑な認知過程を評価するのに対し、関連論文は生理的指標を組み合わせることでランニング効率を客観的に評価している。性能指標も、基準論文では回答時間、正確性、N2成分のピーク時間と大きさであるのに対し、関連論文ではピッチ数、心拍数、酸素摂取量となっている。これらの違いは、対象とするスポーツの特性と研究目的の違いを反映していると言える。",
  "result_comparison": "基準論文の実験1では、サッカー経験の有無による回答時間や認識できる選手数に有意差は認められなかったものの、自己位置をそのままにした